# The Boston Housing Dataset

---

### Colab Note

Don't forget that you can link your notebook to your drive and save your work there. Then you can download and backup your models, reload them to keep training them, or upload datasets to your drive. 

```python
import os
import sys

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('drive/My Drive/') # 'My Drive' is the default name of Google Drives
    os.listdir()
    
# use os.chdir("my-directory") # to change directory, and
# os.listdir()                 # to list its contents
# os.getcwd()                  # to get the name of the current directory
# os.mkdir("my-new-dir")       # to create a new directory
# See: https://realpython.com/working-with-files-in-python/

# You can also use bash commands directly, preceded by a bang
# !ls
# However, the following will *not* change the Python directory 
# the notebook points to (use os.chdir for that)!
# !cd my-directory
```

---

## 1. Practice

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

### For reproducible results

In Keras ([source](https://keras.io/examples/keras_recipes/reproducibility_recipes/)):
```python
tf.keras.utils.set_random_seed(812) # See below

# If using TensorFlow, this will make GPU ops as deterministic as possible,
# but it will affect the overall performance, so be mindful of that.
tf.config.experimental.enable_op_determinism()
```

Note: `tf.keras.utils.set_random_seed` will do the following ([source](https://github.com/keras-team/keras/blob/f6c4ac55692c132cd16211f4877fac6dbeead749/keras/src/utils/rng_utils.py#L10)):

```python
import random
random.seed(42)

import numpy as np
np.random.seed(42)

tf.random.set_seed(42) # can be any number
```

In [ ]:
(train_data, train_targets), (test_data, test_targets) = tf.keras.datasets.boston_housing.load_data()

### Normalization

Compute `mean` and `std` from `train_data`, and apply those to both `train_data` and `test_data`.

In [ ]:
# compute the mean, subtract from `train_data`
# then the std, and divide `train_data` by it

# apply the same computation to `test_data` using `mean` and `std` computed above


### Model building & callback

#### Note

Can you make the function below more modular? You could modify it so that it accepts arguments changing the architecture of the network, and other hyperparameters.

In [ ]:
def build_model(clear=True):
    if clear:
        tf.keras.backend.clear_session()
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input((train_data.shape[1],)))
    model.add(tf.keras.layers.Dense(64, activation = 'relu'))
    model.add(tf.keras.layers.Dense(64, activation = 'relu'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(
        optimizer='rmsprop',
        loss='mse',
        metrics=['mae']
    )
    return model

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, epochs):
        super(tf.keras.callbacks.Callback, self).__init__()
        self.epochs = epochs
    def on_epoch_begin(self, epoch, logs=None):
        c = ['|', '/', '-', '\\'] 
        print(f"\r{c[epoch % 4]} epoch: {epoch+1}/{self.epochs}", end="")
    def on_train_end(self, logs=None):
        print()

### The K-fold algorithm

In [ ]:
K = 4
num_val_samples = len(train_data) // K
num_epochs = 100
all_mae_histories = []

for i in range(K):
    print('processing fold', i)
    
    # Prepare the validation data: data from partition i
    a, b =        # prepare indices
    val_data =    # extract validation samples from train_data
    val_targets = # extract validation targets from train_targets
    
    # Prepare the training data: data from all other partitions
    partial_train_data =    # extract partial train samples from train_data
    partial_train_targets = # # extract partial train samples from  train_targets

    # Build the Keras model (already compiled)
    model = # use your function
    
    # Train the model using .fit()
    # - train: partial_train_data, partial_train_targets
    # - validation: val_data, val_targets
    # - number of epochs, batch_size=1 for now
    # - silent mode recommended, verbose=0
    # collect the returned object in `history`

    mae_history = # collect val_mae from history
    # add mae_history to all_mae_histories

### Visualise your results

Can you think of a way to automate the visualisation once the training is done? This would mean encapsulating the plotting code into a function, and calling it once the K-fold loop is done.

In [ ]:
average_mae_history = np.array(all_mae_histories).mean(axis=0)
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
def smooth_curve(points, beta = 0.9):       # beta must be between 0 and 1!
    smoothed_points = []
    for current in points:
        if smoothed_points:                 # (an nonempty list is 'True')
            previous = smoothed_points[-1]  # the last appended point
                                            # ↓ a weighted sum of previous & point, controlled by beta
            smoothed_points.append(beta * previous + (1 - beta) * current)
        else:
            smoothed_points.append(current) # at the start, the list is empty, we just add the first point
    return smoothed_points

In [ ]:
smooth_mae_history = smooth_curve(average_mae_history[10:])
plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

### Experiments

- Run k-fold validation on the Boston dataset;
- At first, we used a mini-batch of 1. Experiment with different mini-batch sizes. What do you observe? Can you account for your observation?
- Run a series of experiments to find the best model, like in previous labs.
- A more advanced experiment could be to implement *iterated K-fold validation with shuffling*, as mentioned in the lecture.

## 2. Conclusion

Retrain a final model (with the hyperparameters that yielded the best performance, up to the best epoch) on the entire the training data (`train_data` and `train_targets`) and evaluate on the test data (`test_data`, `test_targets`).

You can now use your model to make predictions on by selecting one data point in `test_data`, and compare the prediction to the equivalent price in `test_targets`.

## 3. Advanced/Optional: the California dataset

In the [`lab-5-CALIFORNIA.ipynb`](https://github.com/jchwenger/AI/blob/main/labs/5-lab/lab-5-CALIFORNIA.ipynb), steps are given to process the data in the California housing dataset from scratch. This is more involved, as the dataset has to be downloaded and processed manually. This counts as a 'not covered' dataset in the first coursework.

### Save and load models

To save and load models locally, you can use [the high-level API](https://www.tensorflow.org/tutorials/keras/save_and_load):
```python
model.save("my_imdb_model.keras")
```
Later one, to reload it, use:
```python
reloaded_model = tf.keras.models.load_model('my_imdb_model.keras')
```

It is also possible to save not just the model, but also the state of your optimiser, and every variable used during training, using the more involved [checkpoints](https://www.tensorflow.org/guide/checkpoint#create_the_checkpoint_objects).